In [1]:
!cat .env

STAGE=CCR_PRODUCTION


In [18]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
import os
from PIL import Image
from io import BytesIO
import datetime
import json
import re
import pandas
import openpyxl
from uuid import uuid4
from helpers.signals import DisableSignals
from helpers.apps.reportings import form_fields_dict
from helpers.strings import get_obj_from_path
from helpers.histories import bulk_update_with_history
from simple_history.utils import bulk_create_with_history

In [2]:
def load_excel(filename,sheetPosision=0):
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[wb.sheetnames[sheetPosision]]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        img={}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value
        data.append(row_dict)
    return data,sheet

In [4]:
def ret_campo_foto_array(classe):
    campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[a['apiName']] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[a['apiName']].append({"inner_api":inner['apiName'],
                                               "main_array":a['displayName'],"inner_display":inner['displayName']})
            campos.append(temp)
    return campos

In [5]:
def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [6]:
def get_photo_kind(kind):
    company=Company.objects.get(name="CCR - Rio Sp")
    valor = next(a['value'] for a in company.custom_options['reportingFile']['fields']['kind']['selectOptions']['options'] if a['name'] == kind)
    return valor

In [7]:
def convert_images_to_jpg(root_dir):
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            filepath = os.path.join(subdir, file)
            if filepath.endswith(".png") or filepath.endswith(".jpeg"):
                with Image.open(filepath) as im:
                    im.convert("RGB").save(filepath.replace(".png", ".jpg").replace(".jpeg", ".jpg"))
                os.remove(filepath)

In [8]:
filename='Marcas_Viárias_Upload_Monitoração_Atualizado'
planilha,_=load_excel(filename, 1)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
# exemplo: 'CCR_Monitoracao/marcas viarias/RJ/Sul/'
# s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/2024/eps_sin_horizontal/Dispositivos/
# s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/dreProf/
caminho_imagens='arquivos_monitoracao/2024/acessos_v2/'
# convert_images_to_jpg(caminho_imagens)s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/2024/ocupacoes/

# Se o caminho utiliza identificação antt e a quantidades maxima de fotos

In [11]:
novo_caminho=False
qtd_fotos_por_linhas = 5

# Somente Fotos Upload

In [21]:
user=User.objects.get(username="rlcs")
company=Company.objects.get(name='CCR - Rio Sp')
# _,sheet=load_excel('Template - Monitoração - Barreira - BR-116', 1)

In [13]:
len(planilha),planilha[0]

(361,
 {'Código Apontamento ': 'CCR-FDO-2024.01449',
  'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.71199',
  'Latitude': -23.138985,
  'Longitude': -45.759716,
  'km': '134+131',
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Status': 'Identificado',
  'Equipe': 'Faixa de Domínio',
  'Encontrado em': None,
  'Executado em': datetime.datetime(2023, 12, 13, 0, 0),
  'Prazo': None,
  'Sentido': 'Sul',
  'Classe': 'Monitoração de Acessos',
  'Faixa': 'Acesso',
  'Rodovia': 'BR-116 SP',
  'Identificação ANTT': 'AC116SP_134+131_PS_01-R0',
  'Lat': -23.138985,
  'Long': -45.759716,
  'Município': 'Sâo José dos Campos',
  'UF do Acesso': 'SP',
  'Nome/Razão Social': 'Município de São José dos Campos',
  'Endereço de Entrega': None,
  'Telefone (Acesso)': None,
  'CEP de Entrega': '12.209-530 ',
  'Telefone de Entrega': None,
  'Celular': None,
  'Folha': None,
  'Matrícula': None,
  'Cartório': None,
  'Tipo de Acesso': 'Público',
  'Tipo 

In [14]:
ret_campo_foto_array(OccurrenceType.objects.get(name=planilha[0]['Classe']))

[{'therapy': [{'inner_api': 'treatmentImages',
    'main_array': 'Ações',
    'inner_display': 'Imagens'}]}]

# Rodar pra erros da planilha por multiples valores retornados no get
# Feito pelo Serial do apontamento

In [16]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha)):
    try:
        ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
        try:
            parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
            rep=Reporting.objects.get(
#                                         parent__number=parent.strip(),
#                                         form_data__id_ccr_antt=ficha.strip(),
                                        number=a['Código Apontamento '],
#                                       km=a['km'],
                              occurrence_type__name=a['Classe'].strip(),company=company)
        except:
#             ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
            rep=Reporting.objects.get(form_data__id_ccr_antt=ficha.strip(),
                                      firm__name=a['Equipe'].strip(),
                                      occurrence_type__name=a['Classe'].strip(),
                                      company=company)
    
#         rep.form_data['id_ccr_antt'] = ficha
        img=[]
#         if '116' in rep.road_name:
#             ficha_path = 'BR-116/' if novo_caminho else ''
#         else:
#             ficha_path = 'BR-101/' if novo_caminho else ''
        ficha_path = ficha.upper()+'/' if novo_caminho else ''
        for i in range(1,qtd_fotos_por_linhas+1):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': ficha_path+a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i))
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,qtd_fotos_por_linhas+1):
#                             print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}")
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind} ||| {key}")
                                uuid = uuid4()
                                indice=i-1 
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"," | ",rep.form_data[key] if rep.form_data.get(key) else None," | ",a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"))
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [''.join(x.keys()) for x in rep.form_data[key]]:
                                    rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                elif i > len(rep.form_data[key]):
                                    rep.form_data[key][-1][convert_string(v['inner_api'])] = [] if not rep.form_data[key][-1].get(convert_string(v['inner_api'])) else rep.form_data[key][-1][convert_string(v['inner_api'])]
                                    rep.form_data[key][-1][convert_string(v['inner_api'])].append(str(uuid))
                                else:
                                    rep.form_data[key][indice][convert_string(v['inner_api'])] = [] if not rep.form_data[key][indice].get(convert_string(v['inner_api'])) else rep.form_data[key][indice][convert_string(v['inner_api'])]
                                    rep.form_data[key][indice][convert_string(v['inner_api'])].append(str(uuid))
#                                 print(f"ADICIONADO {rep.form_data[key]}")
                                img.append({
                                    'foto': ficha_path+a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
                    if len(value) >1 and rep.form_data.get(key):
                        my_dict={}
#                         rep.form_data[key] = [{d} for d in rep.form_data.get(key)]
                        for d in rep.form_data.get(key):
                            my_dict.update(d)
                        rep.form_data[key] = [my_dict]

#         print(rep.number,rep.form_data)
        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),i+2,e.r)
        erros.append(a)

  0%|          | 0/361 [00:00<?, ?it/s]

In [17]:
len(erros),len(objects),objects[0],objects[0][0].__dict__

(0,
 361,
 (<Reporting: [CCR - Rio Sp] CCR-FDO-2024.01449 - 2024-02-21 19:47:43+00:00>,
  [{'foto': 'AC116SP_134+131_PS_01_R0 (1)',
    'data_foto': None,
    'tipo_foto': None,
    'des_foto': None},
   {'foto': 'AC116SP_134+131_PS_01_R0 (2)',
    'data_foto': None,
    'tipo_foto': None,
    'des_foto': None}]),
 {'_state': <django.db.models.base.ModelState at 0x7f27d2336c50>,
  'uuid': UUID('9a935028-5158-4c46-947d-5d0d155fa237'),
  'number': 'CCR-FDO-2024.01449',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-116 SP',
  'road_id': 1,
  'km': 134.131,
  'end_km': 134.131,
  'km_reference': None,
  'project_km': None,
  'project_end_km': None,
  'point': <Point object at 0x7f27d2311be0>,
  'direction': '1',
  'lane': '51',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('709eb517-9d0d-43cd-b8aa-9bc84bcd020a'),
  'firm_id': UUID('cc93f857-d490-4761-9f4e-ce15c4cf4173'),
  'occurrence_type_id': UUID('4733f18d-34eb-4802-8e44-d

In [18]:
extensoes=['.JPG','.jpeg','.png']
apt=[]
fotos_apt=[]
for reporting, images in tqdm(objects):
    can_save=False
    
    for image in images:
        
        foto_file = image.get('foto') if any(image.get('foto').lower().endswith(ext) for ext in ['.JPG','.jpeg','.png','.txt','.jpg','.wmf','.pdf']) else image.get('foto') + '.jpg'
#         print(foto_file,' | ',caminho_imagens+foto_file)
        if ReportingFile.objects.filter(reporting=reporting,
                                        upload__icontains=foto_file.replace('.jpg','').replace('.jpeg','').replace('.png','')).exists() or image.get('foto') == '':
            continue
        reporting_file = ReportingFile(
            uuid= image.get('uuid_foto',uuid4()),
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else '',
            datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            is_shared=False,
            point=reporting.point,
            upload=caminho_imagens+foto_file
        )

        try:
            reporting_file.upload.size
            fotos_apt.append(reporting_file)
#             reporting_file.save()
#             print(reporting_file.upload,reporting_file.uuid)
            can_save=True
        except Exception as e:
            print('Não encontrado1: ',reporting_file.upload)
            for ext in extensoes:
                try:
                    reporting_file.upload = caminho_imagens+foto_file.replace('.jpg',ext)
                    reporting_file.upload.size
                    fotos_apt.append(reporting_file)
#                     reporting_file.save()
                    can_save=True
                    break
                except:
                    print('Não encontrado2: ',reporting_file.upload)
                    continue
    if can_save:
        print('append',reporting.number)
        apt.append(reporting)
#         with DisableSignals():
#             reporting.save()

  0%|          | 0/361 [00:00<?, ?it/s]

append CCR-FDO-2024.01449
append CCR-FDO-2024.01450
append CCR-FDO-2024.01451
append CCR-FDO-2024.01452
append CCR-FDO-2024.01453
append CCR-FDO-2024.01454
append CCR-FDO-2024.01455
append CCR-FDO-2024.01456
append CCR-FDO-2024.01457
append CCR-FDO-2024.01458
append CCR-FDO-2024.01459
append CCR-FDO-2024.01460
append CCR-FDO-2024.01461
append CCR-FDO-2024.01462
append CCR-FDO-2024.01463
append CCR-FDO-2024.01464
append CCR-FDO-2024.01465
append CCR-FDO-2024.01466
append CCR-FDO-2024.01467
append CCR-FDO-2024.01468
append CCR-FDO-2024.01469
append CCR-FDO-2024.01470
append CCR-FDO-2024.01471
append CCR-FDO-2024.01472
append CCR-FDO-2024.01473
append CCR-FDO-2024.01474
append CCR-FDO-2024.01475
append CCR-FDO-2024.01476
append CCR-FDO-2024.01477
append CCR-FDO-2024.01478
append CCR-FDO-2024.01479
append CCR-FDO-2024.01480
append CCR-FDO-2024.01481
append CCR-FDO-2024.01482
append CCR-FDO-2024.01483
append CCR-FDO-2024.01484
append CCR-FDO-2024.01485
append CCR-FDO-2024.01486
append CCR-F

append CCR-FDO-2024.01771
append CCR-FDO-2024.01772
append CCR-FDO-2024.01773
append CCR-FDO-2024.01774
append CCR-FDO-2024.01775
append CCR-FDO-2024.01776
append CCR-FDO-2024.01777
append CCR-FDO-2024.01778
append CCR-FDO-2024.01779
append CCR-FDO-2024.01780
append CCR-FDO-2024.01781
append CCR-FDO-2024.01782
append CCR-FDO-2024.01785
append CCR-FDO-2024.01783
append CCR-FDO-2024.01784
append CCR-FDO-2024.01786
append CCR-FDO-2024.01787
append CCR-FDO-2024.01788
append CCR-FDO-2024.01789
append CCR-FDO-2024.01790
append CCR-FDO-2024.01791
append CCR-FDO-2024.01792
append CCR-FDO-2024.01793
append CCR-FDO-2024.01794
append CCR-FDO-2024.01795
append CCR-FDO-2024.01796
append CCR-FDO-2024.01797
append CCR-FDO-2024.01798
append CCR-FDO-2024.01799
append CCR-FDO-2024.01800
append CCR-FDO-2024.01801
append CCR-FDO-2024.01802
append CCR-FDO-2024.01803
append CCR-FDO-2024.01804
append CCR-FDO-2024.01805
append CCR-FDO-2024.01806
append CCR-FDO-2024.01807
append CCR-FDO-2024.01808
append CCR-F

In [19]:
len(fotos_apt),len(apt)

(712, 356)

In [22]:
fot = bulk_create_with_history(fotos_apt, ReportingFile, default_user=user)
apontamentos = bulk_update_with_history(apt,Reporting, use_django_bulk=True,batch_size=80,user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)


In [86]:
reporting.form_data

{'uf': 'RJ',
 'cpf': '2332424',
 'lat': 1,
 'city': 'Itaguaí',
 'long': 2,
 'name': 'jose renato',
 'zone': '1',
 'sheet': 1,
 'state': '1',
 'address': 'rua rj',
 'cartory': '3',
 'celular': '33333333333',
 'therapy': [{'action_date': '2023-11-21T17:22:00.000Z',
   'description': 'abc',
   'action_number': 1,
   'occurrence_type': '66efad60-cf98-4807-9755-5ecbfbb2c02e',
   'treatment_images': ['2d0d73df-b651-4155-af17-e50668526eba']}],
 'matricule': 2,
 'ocupation': '7',
 'situation': '1',
 'telephone': '2222222222',
 'owner_fone': '1111111111',
 'id_ccr_antt': 'OC 101 RJ 400+000 S ',
 'postal_code': '0000000000',
 'qual_fx_dom': 'yyy',
 'city_of_prop': 'rj',
 'observations': '123',
 'titular_kind': '2',
 'lane_occupied': '1',
 'lane_extension': 1,
 'ocupation_kind': '6',
 'characteristics': '3',
 'ocupation_others': 'aa',
 'ocupation_kind_others': 'xx',
 'utilization_condition': '8',
 'lane_extension_not_edified': 2,
 'lane_extension_edified_area': 3,
 'utilization_condition_other': 

In [92]:
reporting=Reporting.objects.get(uuid='221b8853-75b4-4d52-a512-9bf2e62a20b6')
caminho_imagens+'OC116SP_058+622_PN_01_R0 (2).jpg'

'arquivos_monitoracao_homolog/OC116SP_058+622_PN_01_R0 (2).jpg'

In [93]:
reporting_file = ReportingFile(
            uuid= uuid4(),
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else '',
            datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            point=reporting.point,
            is_shared=False,
            upload=caminho_imagens+'OC116SP_058+622_PN_01_R0 (2).jpg'
        )

In [94]:
reporting_file.save()

In [95]:
reporting_file.uuid


UUID('18e8ecd0-d9ca-450c-b3b4-3e29839ece87')

In [13]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha[4541:])):
    try:
        ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Código Jusante')
        try:
            parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a.get('Código do Inventário para vínculo com apontamento','')
            rep=Reporting.objects.get(parent__number=parent.strip(),
#                                       km=a['km'],
                                      occurrence_type__name=a['Classe'].strip(),company=company)
        except:
#             print(ficha)
            rep=Reporting.objects.get(occurrence_type__name=a['Classe'].strip(),company=company
                                      ,form_data__code=ficha.strip()
#                                       ,parent__number=parent.strip()
#                 ,form_data__id_ccr_antt=ficha.strip()
                                     )
    
        img=[]

        ficha_path = ficha+'/' if novo_caminho else ''
        for i in range(1,qtd_fotos_por_linhas+1):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': ficha_path+a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i))
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,qtd_fotos_por_linhas+1):
#                             print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}")
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
                                uuid = uuid4()
                                indice=i-1
                                #funcionando com um erro  
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"," | ",rep.form_data[key]," | ",a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"))
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [list(teste.keys()) for teste in rep.form_data[key]]:
                                    rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                else:
                                    rep.form_data[key][indice][convert_string(v['inner_api'])] = [] if not rep.form_data[key][indice].get(convert_string(v['inner_api'])) else rep.form_data[key][indice][convert_string(v['inner_api'])]
                                    rep.form_data[key][indice][convert_string(v['inner_api'])].append(str(uuid))

                                img.append({
                                    'foto': ficha_path+a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
                    if len(value) >1 and rep.form_data.get(key):
                        my_dict={}
                        for d in rep.form_data.get(key):
                            my_dict.update(d)
                        rep.form_data[key] = [my_dict]


        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),i+2,e,rep)
        erros.append(a)

  0%|          | 0/2412 [00:00<?, ?it/s]

In [14]:
a['Classe']

'Monitoração Drenagem Profunda Elemento'

In [15]:
# import pandas as pd
# pd.DataFrame(data=erros).to_excel(filename+'_erros.xlsx')

In [30]:
foto_file,reporting_file.upload,image.get('foto')

('.jpg', <FieldFile: arquivos_monitoracao/dreProf/.jpg>, '')

In [18]:
# for reporting, images in tqdm(objects):
#     for image in images:
#         image['foto'] = image.get('foto').replace('OAEC','OAE')

  0%|          | 0/533 [00:00<?, ?it/s]

In [15]:
objects[0],objects[0][0].__dict__

((<Reporting: [CCR - Rio Sp] CCR-OAC-2023.06574 - 2022-10-18 03:00:00+00:00>,
  [{'foto': '1_DA 101 RJ 382+283 N 1_foto1.jpg',
    'data_foto': datetime.datetime(2022, 10, 18, 0, 0),
    'tipo_foto': 'Outro',
    'des_foto': 'Panorâmica'},
   {'foto': '1_DA 101 RJ 382+283 N 1_foto2.jpg',
    'data_foto': datetime.datetime(2022, 10, 18, 0, 0),
    'tipo_foto': 'Outro',
    'des_foto': 'Detalhe'}]),
 {'_state': <django.db.models.base.ModelState at 0x7f9dbf3e2590>,
  'uuid': UUID('b6b4ebab-345f-48c9-a0a0-0f853a0347c1'),
  'number': 'CCR-OAC-2023.06574',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-101 RJ',
  'road_id': 2040,
  'km': 382.371,
  'end_km': 382.371,
  'km_reference': None,
  'project_km': 0.0,
  'project_end_km': 0.0,
  'point': <Point object at 0x7f9dbf436370>,
  'direction': '0',
  'lane': '99',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('a08c9011-

In [29]:
u='arquivos_monitoracao/terra/101/TP 101 RJ 389+900 S1/Foto 22-11-2022 10.33.07'
len(u)

76

In [83]:
rep=Reporting.objects.filter(parent__number='CCR-Inv-2023.61752')
# rep.parent = Reporting.objects.get(uuid='00169b3d-9a09-488b-98c7-762760df4efa')
rep

<QuerySet [<Reporting: [CCR - Rio Sp] CCR-EPS-2023.06483 - 2022-11-16 03:00:00+00:00>, <Reporting: [CCR - Rio Sp] CCR-EPS-2023.07093 - 2022-11-23 03:00:00+00:00>]>

In [82]:
erros[0]

{'km': 233.484,
 'km final': 233.489,
 'Status': 'Identificado',
 'Equipe': 'Civil: SGS',
 'Encontrado em': datetime.datetime(2022, 11, 23, 0, 0),
 'Executado em': datetime.datetime(2022, 11, 23, 0, 0),
 'Sentido': 'Canteiro Central',
 'Classe': 'Monitoração de Defensas Metálicas',
 'Faixa': 'Não se aplica',
 'Rodovia': 'BR-116 RJ',
 'Identificação ANTT': 'DM 116 RJ 233+484 CC 2',
 'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.61752',
 'Latitude': -22.666961,
 'Longitude': -43.842898,
 'X UTM': 573233.31,
 'Y UTM': 7513422.12,
 'Zona UTM': '23K',
 'Comprimento': 4,
 'Altura Média': 0.68,
 'Dispositivo Pontual': None,
 'Tipo de dispositivo pontual': None,
 'Poste Metálico': 'NÃO',
 'Dispositivo Longitudinal': None,
 'Tipo de dispositivo longitudinal': None,
 'Terminal Abatido': 'NÃO',
 'Defensa Defletida': 'NÃO',
 'Absorvedor de Energia': 'NÃO',
 'Aéreo': 'NÃO',
 'Ancoragem em Talude': 'NÃO',
 'Presença De Elemento Refletivo': 'SIM',
 'Tipo de Refletores': 'Monodire

In [45]:
rep.form_data

{'uf': 'UF',
 'index': 1,
 'notes': 'Observações',
 'x_utm': 0.04,
 'y_utm': 0.88,
 'length': 0.16,
 'therapy': [{'treatment_images': ['91882ee6-5802-4df2-8735-241c0e0a704a']}],
 'distance': 0.12,
 'latitude': 0.14,
 'zona_utm': 'Zona UTM',
 'longitude': 0.3,
 'contention': [{'inner_pictures': ['8ccef38a-aa8c-44bd-9ee9-a926b0c407dd']}],
 'risk_level': '4',
 'vegetation': '4',
 'id_ccr_antt': 'Identificação ANTT',
 'inclination': 0.29,
 'dre_kind_sup': '2',
 'rupture_case': False,
 'is_contention': False,
 'water_presence': '6',
 'ocorrencia_tipo': '4',
 'probable_causes': '8',
 'tipo_terrapleno': '1',
 'condition_dre_sup': '3',
 'drainage_pictures': [{'drainage_picture': ['0e188611-8125-4330-a0ed-3d1b73a9f28e']}],
 'height_terrapleno': 0.82,
 'passivo_ambiental': False,
 'terrain_embossing': '2',
 'condition_dre_prof': '3',
 'mandatory_pictures': [],
 'vegetation_density': '1',
 'drenagem_subterranea': '3',
 'drenagem_superficial': '3',
 'occurrence_type_other': 'Tipo de Ocorrência (Ou

In [14]:
# rep[0].form_data['mandatory_pictures']= rep[0].form_data['mandatoryPictures']
# rep[0].save()

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ


In [38]:
apont = rep

In [40]:
apont.form_data['mandatory_pictures']=[]

In [41]:
apont

<Reporting: [CCR - Rio Sp] CCR-TER-2023.02033 - 2023-09-28 14:51:48.205597+00:00>

In [42]:
# apont.save()

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP


In [11]:
reps=ReportingFile.history.filter(reporting__occurrence_type=rep.occurrence_type,upload__icontains='png',history_type='-')
reps.count()

147

In [22]:
repfile=ReportingFile.objects.filter(reporting__occurrence_type=rep.occurrence_type)

In [23]:
for a in erros:
    try:
        a.upload.size
    except:
        a.delete()

In [46]:
for a in tqdm(erros):
        parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
        rep=Reporting.objects.filter(parent__number=parent.strip(),
                          occurrence_type__name=a['Classe'].strip(),company=company).first()
#         rep.delete()

In [76]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha)):
    try:
        try:
            parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
            rep=Reporting.objects.get(parent__number=parent.strip(),
                                      km=a['km'],
                              occurrence_type__name=a['Classe'].strip(),company=company)
        except:
            ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Ficha')
            rep=Reporting.objects.get(form_data__id_ccr_antt=ficha.strip(),
                              occurrence_type__name=a['Classe'].strip(),company=company)
    
        img=[]
        for i in range(1,11):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i))
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,11):
                            print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}")
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
    #                             print(a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{i}"))
                                uuid = uuid4()

                                #funcionando com um erro    
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [list(teste.keys())[0] for teste in rep.form_data[key]]:
                                    rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                else:
                                    rep.form_data[key][-1][convert_string(v['inner_api'])].append(str(uuid))

                                img.append({
                                    'foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
                    if len(value) >1 and rep.form_data.get(key):
                        my_dict={}
                        for d in rep.form_data.get(key):
                            my_dict.update(d)
                        rep.form_data[key] = [my_dict]


        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),i+2,e,rep)
        erros.append(a)

  0%|          | 0/6 [00:00<?, ?it/s]

Anomalias_1: Fotos - Foto_1
Anomalias_1: Fotos - Foto_2
Anomalias_1: Fotos - Foto_3
Anomalias_1: Fotos - Foto_4
Anomalias_1: Fotos - Foto_5
Anomalias_1: Fotos - Foto_6
Anomalias_1: Fotos - Foto_7
Anomalias_1: Fotos - Foto_8
Anomalias_1: Fotos - Foto_9
Anomalias_1: Fotos - Foto_10
Imagens da Monitoração_1: Iluminação Diurna - Foto_1
Imagens da Monitoração_1: Iluminação Diurna - Foto_2
Imagens da Monitoração_1: Iluminação Diurna - Foto_3
Imagens da Monitoração_1: Iluminação Diurna - Foto_4
Imagens da Monitoração_1: Iluminação Diurna - Foto_5
Imagens da Monitoração_1: Iluminação Diurna - Foto_6
Imagens da Monitoração_1: Iluminação Diurna - Foto_7
Imagens da Monitoração_1: Iluminação Diurna - Foto_8
Imagens da Monitoração_1: Iluminação Diurna - Foto_9
Imagens da Monitoração_1: Iluminação Diurna - Foto_10
Imagens da Monitoração_1: Iluminação Noturna - Foto_1
Imagens da Monitoração_1: Iluminação Noturna - Foto_2
Imagens da Monitoração_1: Iluminação Noturna - Foto_3
Imagens da Monitoração_1: 

In [79]:
objects[4],objects[4][0].__dict__

((<Reporting: [CCR - Rio Sp] CCR-DEF-2023.11273 - 2023-11-28 19:28:22+00:00>,
  [{'foto': 'PI 101 RJ 397+280 S 1 RF foto1',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('f905d104-389a-432d-8410-680722491ac1')},
   {'foto': 'PI 101 RJ 397+280 S 1 RF foto2',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('88570876-b040-457d-b948-8a96f05aa66e')},
   {'foto': 'PI 101 RJ 397+280 S 1 RF foto3',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('d0b10d13-fd41-4b5d-8a4b-1382bbe270be')}]),
 {'_state': <django.db.models.base.ModelState at 0x7fcf883556d0>,
  'uuid': UUID('3693020e-16ca-4318-92ab-8b039216977c'),
  'number': 'CCR-DEF-2023.11273',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-101 RJ',
  'road_id': 2040,
  'km': 397.28,
  'end_km': 397.6,
  'km_reference': None,
  'project_km': None,
  'project_end_km': None,

In [55]:
rep.form_data.keys(),key,rep.form_data

(dict_keys(['uf', 'city', 'latitude', 'longitude', 'tipoposte', 'id_ccr_antt', 'modeloposte', 'qtdlampadas', 'tipolampada', 'latitude_end', 'longitude_end', 'qtdluminarias', 'tipoiluminacao', 'atendimentoabnt', 'modeloluminaria', 'potencialampada', 'qtdpostes_monit', 'general_conservation_state', 'fotos_monitoracao']),
 'fotos_monitoracao',
 {'uf': 'Rio de Janeiro',
  'city': 'ITAGUAÍ',
  'latitude': -22.86627016,
  'longitude': -43.7504935,
  'tipoposte': '1',
  'id_ccr_antt': 'PI 101 RJ 397+830 NS 1',
  'modeloposte': 'Poste De Concreto Circular 9x200dan ',
  'qtdlampadas': 22.0,
  'tipolampada': '1',
  'latitude_end': -22.86627016,
  'longitude_end': -43.7504935,
  'qtdluminarias': 22.0,
  'tipoiluminacao': '5',
  'atendimentoabnt': '1',
  'modeloluminaria': '2',
  'potencialampada': '1',
  'qtdpostes_monit': 22,
  'general_conservation_state': '1',
  'fotos_monitoracao': [{'imagem_noturna': ['dbaf4139-ca3c-4bae-9410-4d289e31146e']}]})

In [52]:
key not in rep.form_data.keys()

False

In [75]:
convert_string(v['inner_api'])

'imagem_noturna'

In [61]:
key

'fotos_monitoracao'

In [58]:
rep.form_data[key]

[{'imagem_noturna': ['dbaf4139-ca3c-4bae-9410-4d289e31146e']}]

In [72]:
[list(teste.keys())[0] for teste in rep.form_data[key]]

['imagem_noturna']

In [74]:
convert_string(v['inner_api']) in [list(teste.keys())[0] for teste in rep.form_data[key]]

True

# EXCLUSÃO DE UUID DE IMAGENS QUE NÃO FORAM VINCULADAS AO APONTAMENTO

In [21]:
from helpers.histories import bulk_update_with_history
from copy import deepcopy
from tqdm.notebook import tqdm

In [22]:
def ret_campo_foto_array_v2(classe):
    campos=[]
    inner_campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[convert_string(a['apiName'])] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[convert_string(a['apiName'])].append({"main_array_display_name":a['displayName'],
                                               "inner_api":convert_string(inner['apiName']),
                                               "inner_display":inner['displayName']})
                    inner_campos.append({convert_string(a['apiName']):convert_string(inner['apiName'])})
            campos.append(temp)
    return campos,inner_campos

In [23]:
company=Company.objects.get(name='CCR - Rio Sp')

In [38]:
rep_edit=[]

for a in tqdm(Reporting.objects.filter(company=company,
                                       number='CCR-FDO-2023.00107')):
    rep_bkp=deepcopy(a)
    #     print(ret_campo_foto_array(a.occurrence_type)[1])
    for b in ret_campo_foto_array_v2(a.occurrence_type)[1]:
        for c,d in b.items():
            # pega a chave do arrayOfObjects se existir
            if a.form_data.get(c):
                # itera sob o arrayOfObjects e o enumera para posterior exclusão
                for i,e in enumerate(a.form_data.get(c)):
                    # se existir a chave da imagem criada no arrayOfObjects e não for 0
                    if e.get(d) is not None and len(e.get(d)) >0:
                        print(e.get(d))
                        # itera sob os uuid de cada imagem e verifica se os reportingFile existe se caso não existe exclui no except
                        for f in e.get(d):
                            try:
                                ReportingFile.objects.get(uuid=f).upload.url
                            except:
                                # remove o uuid que não existe da lista do innerImageArray
                                e.get(d).remove(f)
                                # se estiver vazio a lista do innerImageArray exclui a chave
                                if len(e.get(d))==0:
                                    del a.form_data[c][i][d]
    if a.form_data != rep_bkp.form_data:
        rep_edit.append(a)


  0%|          | 0/1 [00:00<?, ?it/s]

['ddd96af4-232b-44ab-93ee-06a23217cd85']


In [36]:
# len(rep_edit),rep_edit[0].form_data,rep_edit[0].number

In [37]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=20,user=User.objects.get(username='rlcs'))

In [65]:
from datetime import datetime

def week_and_day_semana(dt):
    dias = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo"]
    isocalendar = dt.isocalendar()
    return isocalendar[0],isocalendar[1],dias[isocalendar[2]]

dt = datetime.now()
# dia = week_and_day_semana(dt)

print(week_and_day_semana(dt))

(2024, 5, 'Quinta-feira')


In [19]:
for a in Reporting.objects.get(number='CCR-Inv-2023.78459').children.all():
    print(a.form_data['id_ccr_antt'])

MV 116 RJ 339+360 S 1
ZE 116 RJ 339+450 S 1


In [16]:
dados,_=load_excel('Marcas_Viárias_Upload_Monitoração_Atualizado',0)

{'Código Apontamento': 'CCR-DEF-2024.01989',
 'Identificação ANTT': 'MV 116 SP 231+120 S 3',
 'Latitude (Decimal)': -23.5251325957,
 'Longitude (Decimal)': -46.5878214687,
 'Ano': 2023,
 'Semestre': 'Segundo Semestre',
 'Tipo de Monitoração': 'Primeira Monitoração',
 'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.76851',
 'km': 231.12,
 'km final': 231.12,
 'Status': 'Executado',
 'Equipe': 'Bureau Veritas - Campo',
 'Encontrado em': datetime.datetime(2024, 1, 14, 0, 0),
 'Executado em': datetime.datetime(2024, 1, 14, 0, 0),
 'Sentido': 'Sul',
 'Classe': 'Retrorrefletância de Marcas Viárias',
 'Faixa': 'Marginal 1',
 'Localização': 'Faixa 1',
 'Rodovia': 'BR-116 SP',
 'Observações': None,
 'Status de Atendimento Geral': 'Aprovado',
 'Valor Mínimo Geral': 100,
 'Foto_1': '4f4015dc-0f6d-4461-a69e-f857a9e36fe7',
 'Adicionar retrorrefletância de longitudinal_1: Tipo de marca viária': 'Seta',
 'Adicionar retrorrefletância de longitudinal_1: Descrição': 'Seta',
 'Adiciona

In [20]:
rep_edit=[]
for a in tqdm(dados[29:]):
#     print(a['Código do Inventário para vinculo com apontamento'])
    rep = Reporting.objects.get(company__name='CCR - Rio Sp',
                                number=a['Código Apontamento'],
                               )
    rep.form_data['inspection_campaign_year'] = 2023
    rep.form_data['inspection_campaign_semester'] = '2'
    rep.form_data['monitoring_kind'] = '2'
    rep_edit.append(rep)

  0%|          | 0/888 [00:00<?, ?it/s]

In [22]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=250,user=user)